In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
from cpmpy import * # pip3 install cpmpy
from cpmpy.solvers import CPM_ortools

In [2]:
# helper function to enumerate all models, for a list of key variables
# with limit
def enumerate_all(model, variables, limit=None):
    model = CPM_ortools(model) # this speeds up repeated solving
    count = 0
    while model.solve():
        print(variables.value())
        model += ~all(variables == variables.value())
        if limit is not None:
            count += 1
            if count > limit:
                break

What I think is the underlying model:

In [3]:
# returns (model, variables)
def manual_type01(size, inputData):
    v = intvar(1,size, shape=size)
    
    m = Model()
    
    # data_pairs: v[i] != v[j]
    for (s,t) in inputData:
        m += [v[s] != v[t]]
    
    # the objective
    m.maximize(max(v))

    return (m,v)

In [4]:
# lets enumerate a few
(m,v) = manual_type01(size=10, inputData=[(1,0), (5,4), (8,6)])
enumerate_all(m,v,limit=10)

[ 1  2  1  1  1  2 10  1  1  1]
[ 1 10  1  1  1  2  2  1  1  1]
[ 1  2  1  1  1  2  1  1 10  1]
[ 1  2  1  1  1  2  1  1  2 10]
[ 1  2  1  1  1  2  2  1  1 10]
[ 1  2  1  1  1  2 10  1  1  2]
[ 1  2  1  1  1  2  3  1  1 10]
[ 1  2  1  1  1  2 10  1  1  3]
[ 1  2  1  1  1  2  4  1  1 10]
[ 1  2  1  1  1  2 10  1  1  4]
[ 1  2  1  1  1  2  5  1  1 10]


In [5]:
data = json.load(open('instances/type01/instance0.json'))
# some manual work...
size = len(data['formatTemplate']['list'])
inputData = [list(dd.values()) for dd in data['inputData']['list']]
print(size, inputData)
(m,v) = manual_type01(size=size, inputData=inputData)
enumerate_all(m,v,limit=10)

10 [[1, 0], [5, 4], [8, 6]]
[ 1  2  1  1  1  2 10  1  1  1]
[ 1 10  1  1  1  2  2  1  1  1]
[ 1  2  1  1  1  2  1  1 10  1]
[ 1  2  1  1  1  2  1  1  2 10]
[ 1  2  1  1  1  2  2  1  1 10]
[ 1  2  1  1  1  2 10  1  1  2]
[ 1  2  1  1  1  2  3  1  1 10]
[ 1  2  1  1  1  2 10  1  1  3]
[ 1  2  1  1  1  2  4  1  1 10]
[ 1  2  1  1  1  2 10  1  1  4]
[ 1  2  1  1  1  2  5  1  1 10]


In [6]:
data = json.load(open('instances/type01/instance1.json'))
# some manual work...
size = len(data['formatTemplate']['list'])
inputData = [list(dd.values()) for dd in data['inputData']['list']]
print(size, inputData)
(m,v) = manual_type01(size=size, inputData=inputData)
enumerate_all(m,v,limit=10)

11 [[2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [4, 1], [7, 1], [10, 1], [3, 2], [5, 2], [7, 2], [10, 2], [5, 3], [7, 3], [9, 3], [10, 3], [5, 4], [6, 4], [6, 5], [8, 7], [9, 7], [10, 7], [10, 8], [10, 9]]
[ 3  1  5  1 11  2  4  2  5  3  4]
[ 2  1 11  1  3  4  1  2  3  3  4]
[ 2  1  3  1  3 11  1  2  3  3  4]
[ 2  1  3  1  4 11  1  2  3  3  4]
[ 2  1  3  1  5 11  1  2  3  3  4]
[ 2  1  3  1  6 11  1  2  3  3  4]
[ 2  1  3  1  7 11  1  2  3  3  4]
[ 4  1 11  1  3  2  1  2  3  3  4]
[ 1 11  5  4  4  3  2  2  3  3  1]
[ 1  4  5  3  3  4  2  2  3 11  1]
[ 1 11  5  3  3  4  2  2  3  4  1]


Example checking the model
...

See Type02, but then with also the data preprocessing like above...

Some tests with learned constraints

In [7]:
from learner import *

In [18]:
    data = json.load(open('instances/type01/instance0.json'))
    size = len(data['formatTemplate']['list'])
    inputData = [d['list'] for d in data['tests']]
    print(inputData[0])
    lb, ub = constraint_learner(inputData[0])

[8, 1, 7, 9, 8, 3, 3, 3, 10, 7]


Trains on single example?

In [15]:
lb, ub = constraint_learner(inputData[0:2])
lb

{'x': [x, x],
 'Abs(x)': [Abs(x), Abs(x)],
 'x**2': [x**2, x**2],
 'Mod(x, 2)': [Mod(x, 2), Mod(x, 2)],
 'x + y': [x + y, x + y, x + y, x + y],
 'x - y': [x - y, x - y, x - y, x - y],
 '-x + y': [-x + y, -x + y, -x + y, -x + y],
 'Abs(x) + Abs(y)': [Abs(x) + Abs(y),
  Abs(x) + Abs(y),
  Abs(x) + Abs(y),
  Abs(x) + Abs(y)],
 'Abs(x) - Abs(y)': [Abs(x) - Abs(y),
  Abs(x) - Abs(y),
  Abs(x) - Abs(y),
  Abs(x) - Abs(y)],
 '-Abs(x) + Abs(y)': [-Abs(x) + Abs(y),
  -Abs(x) + Abs(y),
  -Abs(x) + Abs(y),
  -Abs(x) + Abs(y)],
 'x**2 + y**2': [x**2 + y**2, x**2 + y**2, x**2 + y**2, x**2 + y**2],
 'x**2 - y**2': [x**2 - y**2, x**2 - y**2, x**2 - y**2, x**2 - y**2],
 '-x**2 + y**2': [-x**2 + y**2, -x**2 + y**2, -x**2 + y**2, -x**2 + y**2],
 'Mod(x, 2) + Mod(y, 2)': [Mod(x, 2) + Mod(y, 2),
  Mod(x, 2) + Mod(y, 2),
  Mod(x, 2) + Mod(y, 2),
  Mod(x, 2) + Mod(y, 2)],
 'Mod(x, 2) - Mod(y, 2)': [Mod(x, 2) - Mod(y, 2),
  Mod(x, 2) - Mod(y, 2),
  Mod(x, 2) - Mod(y, 2),
  Mod(x, 2) - Mod(y, 2)],
 '-Mod(x, 2

Does not work...

I would have expected it to take a numpy array of examples and to operate on the columns:

In [36]:
sols = np.array([d['list'] for d in data['solutions']])
objs = np.array([[d['objective']] for d in data['solutions']]) # vertical, watch the []s
sols

array([[ 4,  5,  2, ...,  7,  4,  7],
       [ 3,  8,  4, ...,  6,  5,  5],
       [ 1,  8,  9, ...,  3,  5, 10],
       ...,
       [ 4,  8, 10, ...,  7,  1,  9],
       [ 3,  4, 10, ...,  5,  3, 10],
       [ 1,  9,  9, ...,  7,  1,  3]])

In [38]:
sols.shape

(1000, 10)

In [52]:
import sympy
sols = np.array([d['list'] for d in data['solutions']])
# try to reconstruct some of Mohit's

x, y = symbols('x y')
n_vars = sols.shape[1] # only works for lists... TODO
learned = dict() # key=str(expr) val = {(vars): (lb,ub)}
for expr in generate_unary_exp(x):
    learned[str(expr)] = dict()
    print(expr)
    for i in range(n_vars):
        #print(sols[:,i])
        #print(expr.subs({x: sols[:,i]})) # does not work
        # Use sympy.lambdify() method
        f = sympy.lambdify(x, expr, "math")
        vals = f(sols[:,i])
        learned[str(expr)][(i)] = (min(vals), max(vals))

learned

x
Abs(x)
x**2
Mod(x, 2)
x
Abs(x)
x**2
Mod(x, 2)


In [78]:
cpvars = []
for vdict in data['formatTemplate']['list']:
    # {'high': 10, 'low': 1, 'type': 'dvar'}
    cpvars.append(intvar(vdict['low'],vdict['high']))
cpvars = cpm_array(cpvars) # make it a CPM/Numpy array

m = Model()
for expr,inst in learned.items():
    for i, (lb,ub) in inst.items():
        # why is lambdify doing 'pow' instead of **?
        if "**" not in expr:
            e = sympy.sympify(expr)
            #cpm_e = e.subs({x: cpvars[i]})
            f = sympy.lambdify(x, e, "math") # not sure about the 'x' part here : /
            cpm_e = f(cpvars[i])
            m += [cpm_e >= lb, cpm_e <= ub]
print(m)
m.solve()
print(m.status())
cpvars.value()

Constraints:
    [IV253 >= 1, IV253 <= 10]
    [IV254 >= 1, IV254 <= 10]
    [IV255 >= 1, IV255 <= 10]
    [IV256 >= 1, IV256 <= 10]
    [IV257 >= 1, IV257 <= 10]
    [IV258 >= 1, IV258 <= 10]
    [IV259 >= 1, IV259 <= 10]
    [IV260 >= 1, IV260 <= 10]
    [IV261 >= 1, IV261 <= 10]
    [IV262 >= 1, IV262 <= 10]
    [IV253 >= 1, IV253 <= 10]
    [IV254 >= 1, IV254 <= 10]
    [IV255 >= 1, IV255 <= 10]
    [IV256 >= 1, IV256 <= 10]
    [IV257 >= 1, IV257 <= 10]
    [IV258 >= 1, IV258 <= 10]
    [IV259 >= 1, IV259 <= 10]
    [IV260 >= 1, IV260 <= 10]
    [IV261 >= 1, IV261 <= 10]
    [IV262 >= 1, IV262 <= 10]
    [(IV253) mod 2 >= 0, (IV253) mod 2 <= 1]
    [(IV254) mod 2 >= 0, (IV254) mod 2 <= 1]
    [(IV255) mod 2 >= 0, (IV255) mod 2 <= 1]
    [(IV256) mod 2 >= 0, (IV256) mod 2 <= 1]
    [(IV257) mod 2 >= 0, (IV257) mod 2 <= 1]
    [(IV258) mod 2 >= 0, (IV258) mod 2 <= 1]
    [(IV259) mod 2 >= 0, (IV259) mod 2 <= 1]
    [(IV260) mod 2 >= 0, (IV260) mod 2 <= 1]
    [(IV261) mod 2 >= 0, (I

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [68]:
[1 <= intvar(0,2)]

[IV137 >= 1]

In [65]:
m

Constraints:
    [True, IV114 <= 10]
    [True, IV115 <= 10]
    [True, IV116 <= 10]
    [True, IV117 <= 10]
    [True, IV118 <= 10]
    [True, IV119 <= 10]
    [True, IV120 <= 10]
    [True, IV121 <= 10]
    [True, IV122 <= 10]
    [True, IV123 <= 10]
    [True, IV114 <= 10]
    [True, IV115 <= 10]
    [True, IV116 <= 10]
    [True, IV117 <= 10]
    [True, IV118 <= 10]
    [True, IV119 <= 10]
    [True, IV120 <= 10]
    [True, IV121 <= 10]
    [True, IV122 <= 10]
    [True, IV123 <= 10]
    [True, (IV114) pow 2 <= 100]
    [True, (IV115) pow 2 <= 100]
    [True, (IV116) pow 2 <= 100]
    [True, (IV117) pow 2 <= 100]
    [True, (IV118) pow 2 <= 100]
    [True, (IV119) pow 2 <= 100]
    [True, (IV120) pow 2 <= 100]
    [True, (IV121) pow 2 <= 100]
    [True, (IV122) pow 2 <= 100]
    [True, (IV123) pow 2 <= 100]
    [True, (IV114) mod 2 <= 1]
    [True, (IV115) mod 2 <= 1]
    [True, (IV116) mod 2 <= 1]
    [True, (IV117) mod 2 <= 1]
    [True, (IV118) mod 2 <= 1]
    [True, (IV119) mod